In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import your CRUD Python module for MongoDB
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Update with your MongoDB username and password
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Fetch the data from MongoDB and create a DataFrame
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column from MongoDB documents to avoid conflicts
df.drop(columns=['_id'], inplace=True)

## Debugging - check the DataFrame structure (optional)
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Fix: Add in Grazioso Salvare’s logo
image_filename = 'my-image.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Fix: Add logo to the layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Hr(),
    
    # Add interactive filter options - for example, a dropdown for dog breeds
    html.Div([
        dcc.Dropdown(
            id='filter-breed',
            options=[{'label': breed, 'value': breed} for breed in df['Breed'].unique()],
            placeholder="Select Dog Breed",
            multi=True,
            style={'width': '50%'}
        ),
        dcc.Dropdown(
            id='filter-rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness Rescue'},
                {'label': 'Disaster Tracking', 'value': 'Disaster Tracking'},
                {'label': 'Individual Tracking', 'value': 'Individual Tracking'}
            ],
            placeholder="Select Rescue Type",
            multi=True,
            style={'width': '50%'}
        ),
    ], style={'padding': '20px'}),
    
    html.Hr(),

    # Interactive DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '400px', 'overflowY': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '10px'},
    ),
    
    html.Br(),
    html.Hr(),

    # Dashboard layout for charts side by side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback function to update the DataTable based on selected filters
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-breed', 'value'),
     Input('filter-rescue-type', 'value')]
)
def update_dashboard(selected_breeds, selected_rescue_types):
    # Build MongoDB query based on filters
    query = {}

    if selected_breeds:
        query['Breed'] = {'$in': selected_breeds}
    
    if selected_rescue_types:
        query['RescueType'] = {'$in': selected_rescue_types}
    
    # Fetch the filtered data from MongoDB
    filtered_df = pd.DataFrame.from_records(db.read(query))
    
    # Drop the '_id' column again after filtering
    filtered_df.drop(columns=['_id'], inplace=True)
    
    # Return the filtered data as a dictionary for the DataTable
    return filtered_df.to_dict('records')


# Callback for the geolocation chart (Map) and other chart updates based on filtered data
@app.callback(
    [Output('graph-id', 'children'),
     Output('map-id', 'children')],
    [Input('datatable-id', 'data')]
)
def update_charts(data):
    # Placeholder logic for chart and geolocation map updates
    # Here you would generate charts such as a bar chart and map visualization using Plotly or Dash Leaflet
    
    # Example for geolocation chart using Plotly
    # Create a simple bar chart as a placeholder
    if data:
        df = pd.DataFrame(data)
        bar_chart = px.bar(df, x='Breed', y='Age', title="Dog Breed vs Age")
    else:
        bar_chart = "No data available"

    # Placeholder for geolocation map (Dash Leaflet)
    geolocation_map = dl.Map([dl.TileLayer(), dl.Marker(position=[30.2672, -97.7431])], center=[30.2672, -97.7431], zoom=10)

    return bar_chart, geolocation_map


# Running the app
if __name__ == '__main__':
    app.run_server(debug=True)
